In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Initialize Notion client
from notion_client import Client

import os
notion = Client(auth=os.getenv("NOTION_API_KEY"))


In [11]:
comments = notion.comments.list(block_id=os.getenv("NOTION_PAGE")).get("results", None)
comments

[{'object': 'comment',
  'id': '1c1592a3-0352-8042-8df2-001d0797d9ae',
  'parent': {'type': 'page_id',
   'page_id': '9e05a709-6962-4a79-9c07-97eaafb9db68'},
  'discussion_id': '1c1592a3-0352-803c-a2aa-001c27dbf2b3',
  'created_time': '2025-03-25T21:47:00.000Z',
  'last_edited_time': '2025-03-25T21:47:00.000Z',
  'created_by': {'object': 'user',
   'id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d'},
  'rich_text': [{'type': 'text',
    'text': {'content': 'What kind of bear is best', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'What kind of bear is best',
    'href': None}]},
 {'object': 'comment',
  'id': '1c2592a3-0352-81cf-9796-001d5b293ed7',
  'parent': {'type': 'page_id',
   'page_id': '9e05a709-6962-4a79-9c07-97eaafb9db68'},
  'discussion_id': '1c1592a3-0352-803c-a2aa-001c27dbf2b3',
  'created_time': '2025-03-26T00:22:00.000Z',
  'last_edite

# Message Store Database

## Setup

In [5]:
import sqlite3

# SQLite database configuration
DATABASE_PATH = os.getenv("DATABASE_PATH", "notion_comments.db")

def init_database():
    """Initialize the SQLite database with the required schema."""
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        
        # Create messages table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS messages (
            id TEXT PRIMARY KEY,
            discussion_id TEXT,
            parent_type TEXT,
            parent_id TEXT,
            created_time TEXT,
            last_edited_time TEXT,
            created_by_id TEXT,
            plain_text TEXT
        )
        ''')
        
        conn.commit()
        conn.close()
        print(f"Database initialized at {DATABASE_PATH}")
    except Exception as e:
        print(f"Error initializing database: {e}")

In [6]:
def store_comment_in_db(comment):
    """Store a comment in the SQLite database."""
    try:
        # Extract required fields from the comment
        comment_id = comment["id"]
        discussion_id = comment.get("discussion_id", "")
        
        # Determine parent type and ID
        parent_type = ""
        parent_id = ""
        if "parent" in comment:
            if "page_id" in comment["parent"]:
                parent_type = "page"
                parent_id = comment["parent"]["page_id"]
            elif "block_id" in comment["parent"]:
                parent_type = "block"
                parent_id = comment["parent"]["block_id"]
        
        # Extract timestamps
        created_time = comment.get("created_time", "")
        last_edited_time = comment.get("last_edited_time", "")
        
        # Extract user ID
        created_by_id = ""
        if "created_by" in comment and "id" in comment["created_by"]:
            created_by_id = comment["created_by"]["id"]
        
        # Extract plain text content
        plain_text = ""
        if "rich_text" in comment and len(comment["rich_text"]) > 0:
            plain_text = comment["rich_text"][0].get("text", {}).get("content", "")
        
        # Connect to database and insert/update the comment
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        
        # Check if comment already exists
        cursor.execute("SELECT id FROM messages WHERE id = ?", (comment_id,))
        exists = cursor.fetchone()
        
        if exists:
            # Update existing comment
            cursor.execute('''
            UPDATE messages SET 
                discussion_id = ?,
                parent_type = ?,
                parent_id = ?,
                created_time = ?,
                last_edited_time = ?,
                created_by_id = ?,
                plain_text = ?
            WHERE id = ?
            ''', (discussion_id, parent_type, parent_id, created_time, last_edited_time, 
                  created_by_id, plain_text, comment_id))
        else:
            # Insert new comment
            cursor.execute('''
            INSERT INTO messages (
                id, discussion_id, parent_type, parent_id, 
                created_time, last_edited_time, created_by_id, plain_text
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (comment_id, discussion_id, parent_type, parent_id, created_time, 
                  last_edited_time, created_by_id, plain_text))
        
        conn.commit()
        conn.close()
        print(f"Comment {comment_id} stored in database")
        return True
    except Exception as e:
        print(f"Error storing comment in database: {e}")
        return False

In [7]:
def get_comments_from_db():
    """Retrieve all comments from the database."""
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM messages")
        comments = cursor.fetchall()
        
        # Convert to list of dictionaries
        columns = ["id", "discussion_id", "parent_type", "parent_id", 
                  "created_time", "last_edited_time", "created_by_id", "plain_text"]
        result = [dict(zip(columns, comment)) for comment in comments]
        
        conn.close()
        return result
    except Exception as e:
        print(f"Error retrieving comments from database: {e}")
        return []

## Usage

In [14]:
init_database()

Database initialized at notion_comments.db


In [15]:
for comment in comments:
    store_comment_in_db(comment)

Comment 1c1592a3-0352-8042-8df2-001d0797d9ae stored in database
Comment 1c2592a3-0352-81cf-9796-001d5b293ed7 stored in database


In [16]:
get_comments_from_db()

[{'id': '1c1592a3-0352-8042-8df2-001d0797d9ae',
  'discussion_id': '1c1592a3-0352-803c-a2aa-001c27dbf2b3',
  'parent_type': 'page',
  'parent_id': '9e05a709-6962-4a79-9c07-97eaafb9db68',
  'created_time': '2025-03-25T21:47:00.000Z',
  'last_edited_time': '2025-03-25T21:47:00.000Z',
  'created_by_id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d',
  'plain_text': 'What kind of bear is best'},
 {'id': '1c2592a3-0352-81cf-9796-001d5b293ed7',
  'discussion_id': '1c1592a3-0352-803c-a2aa-001c27dbf2b3',
  'parent_type': 'page',
  'parent_id': '9e05a709-6962-4a79-9c07-97eaafb9db68',
  'created_time': '2025-03-26T00:22:00.000Z',
  'last_edited_time': '2025-03-26T00:22:00.000Z',
  'created_by_id': '28a90a7f-0d89-4566-b10c-65c923c42d6e',
  'plain_text': "AI Assistant: It's a matter of opinion, but many people believe that the best kind of bear is the grizzly bear due to its impressive size and strength. However, pandas are also beloved for their cute appearance and gentle nature. Ultimately, the best ki

# Webhooks: Comment Created

In [4]:
# This is what an event emitted to the server from Notion looks like
example_event = {'id': 'fa230122-3000-47ee-b54b-f32d94ff9938', 'timestamp': '2025-03-31T21:29:49.286Z', 'workspace_id': 'e83f67a4-b5c2-45ad-90ad-f8e8267e1123', 'workspace_name': "Ethan's Notion", 'subscription_id': '1c7d872b-594c-8182-8b10-009956303ceb', 'integration_id': '15fd872b-594c-81d1-9210-00378dd09e38', 'authors': [{'id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d', 'type': 'person'}], 'attempt_number': 1, 'type': 'comment.created', 'entity': {'id': '1c7592a3-0352-8045-900e-001d5e0f8de8', 'type': 'comment'}, 'data': {'page_id': 'd2025568-688b-46fe-a462-9e298aa22c03', 'parent': {'id': 'd2025568-688b-46fe-a462-9e298aa22c03', 'type': 'page'}}}
print(example_event)

{'id': 'fa230122-3000-47ee-b54b-f32d94ff9938', 'timestamp': '2025-03-31T21:29:49.286Z', 'workspace_id': 'e83f67a4-b5c2-45ad-90ad-f8e8267e1123', 'workspace_name': "Ethan's Notion", 'subscription_id': '1c7d872b-594c-8182-8b10-009956303ceb', 'integration_id': '15fd872b-594c-81d1-9210-00378dd09e38', 'authors': [{'id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d', 'type': 'person'}], 'attempt_number': 1, 'type': 'comment.created', 'entity': {'id': '1c7592a3-0352-8045-900e-001d5e0f8de8', 'type': 'comment'}, 'data': {'page_id': 'd2025568-688b-46fe-a462-9e298aa22c03', 'parent': {'id': 'd2025568-688b-46fe-a462-9e298aa22c03', 'type': 'page'}}}


## Use the event data to get the comment that triggered the event

In [7]:
comments = notion.comments.list(block_id=example_event['parent']['id'])
print(comments)

{'object': 'list', 'results': [{'object': 'comment', 'id': '1c7592a3-0352-8045-900e-001d5e0f8de8', 'parent': {'type': 'page_id', 'page_id': 'd2025568-688b-46fe-a462-9e298aa22c03'}, 'discussion_id': '1c7592a3-0352-80cf-a0e6-001c253caabc', 'created_time': '2025-03-31T21:29:00.000Z', 'last_edited_time': '2025-03-31T21:29:00.000Z', 'created_by': {'object': 'user', 'id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d'}, 'rich_text': [{'type': 'text', 'text': {'content': 'THis is a new comment', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'THis is a new comment', 'href': None}]}], 'next_cursor': None, 'has_more': False, 'type': 'comment', 'comment': {}, 'request_id': 'ef832243-db1b-4fa8-976a-465ed60ef376'}


In [10]:
triggering_comment = [comment for comment in comments['results'] if comment['id'] == example_event['entity']['id']]
print(triggering_comment)

[{'object': 'comment', 'id': '1c7592a3-0352-8045-900e-001d5e0f8de8', 'parent': {'type': 'page_id', 'page_id': 'd2025568-688b-46fe-a462-9e298aa22c03'}, 'discussion_id': '1c7592a3-0352-80cf-a0e6-001c253caabc', 'created_time': '2025-03-31T21:29:00.000Z', 'last_edited_time': '2025-03-31T21:29:00.000Z', 'created_by': {'object': 'user', 'id': '59164cd3-f8de-49dc-a432-e8cb48b4a39d'}, 'rich_text': [{'type': 'text', 'text': {'content': 'THis is a new comment', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'THis is a new comment', 'href': None}]}]


## Event handling:

Three kinds of comment events are available for subscription. The type of the event emitted is provided in the event payload under the `type` keyword
1. a new comment was created `type: comment.created`
2. a comment was updated `type: comment.updated`
3. a comment was deleted `type: comment.deleted`

In [14]:
# Creating an OpenAI client



NameError: name 'OpenAI' is not defined

### Event type 1: Comment created
We want to create a new thread using the Openai Assistant's SDK and start a run 

In [ ]:
from openai import OpenAI

client



### Event type 2: Comment updated
We want to check with openAI's assistant SDK to see if there is a thread available for this comment, based on the discussion ID. 